In [1]:
using Pipe
push!(LOAD_PATH, "../word-embeddings2")
using WordEmbeddings
we = @pipe load_embeddings("../word-embeddings2/word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt") |> WE(_...);
using PyCall



In [64]:
using Iterators
@pyimport nltk.corpus as nltk_corpus
n_training = 1000*1000*1000 #4261
training_sents = @pipe nltk_corpus.brown[:sents]() |> filter(s->1<length(s)<=1000, _) |> take(_,n_training)  |> collect |> convert(Vector{Vector{String}},_);


In [66]:
xs = [eval_word_embeddings(we, sent,false)' for sent in training_sents];
#xs = [xs[i] for i in 1:length(xs)]
xs = vcat(xs...)
xs = [xs for _ in 1]


1-element Array{Any,1}:
 1160865x50 Array{Float64,2}:
 -0.38772    -0.107298    -0.672488   …  -1.02026    -0.1513      0.560045  
 -0.0628525   0.141944     0.104873      -0.179453   -0.0565538  -0.0636056 
  0.0407688   0.21597     -0.0910899     -0.454284   -0.295706    0.0266855 
 -0.545411   -0.0638687    0.329359      -0.589683   -0.409102   -0.239369  
 -0.109801   -0.0648981    0.0471347     -0.344632   -0.022939    0.159325  
 -0.312721    0.258346    -0.201343   …  -0.689054    0.293119   -0.76101   
 -0.462468   -0.395314    -0.124963      -0.199847    0.174866    0.0948898 
 -0.609122   -0.172804    -0.275776       0.0410881   0.899009    0.209755  
 -0.0611061   0.00622122  -0.0634503     -0.482041    0.013599   -0.829661  
 -0.36153    -0.127324    -0.682374      -0.458765    0.0512498  -0.00738179
  0.0398781   0.048215     0.0176077  …   0.0252652   0.0304082   0.0337324 
 -0.13188    -0.327422    -0.187588      -0.723885   -0.0257599   0.0164135 
 -0.418452    0.266674

In [67]:
@pyimport numpy
numpy.save("xs_full1", xs)

In [ ]:
@pyimport hmmlearn.hmm as hl_hmm

In [ ]:
hmm = hl_hmm.GMMHMM(n_components=6, n_mix=10, covariance_type="full", n_iter=10)

In [ ]:
hmm[:fit](xs)

In [ ]:
wes, states = hmm[:sample](10)

In [45]:
wes = readdlm("sample.txt", ' ')

10x50 Array{Float64,2}:
 -0.352444   0.086081   -0.289378    -0.0790635  …  -0.260944     0.0587249
 -0.388441  -0.0933392   0.00259688   0.251441       0.0343461    0.175288 
 -1.16707    0.337869    0.359448    -0.130968      -0.332349     0.0554964
 -0.549037  -0.185069    0.0145031   -0.066057      -0.00115987   0.0490553
  0.055385  -0.142676   -0.0251191    0.028343      -0.219891     0.148794 
 -0.279145   0.125427   -0.454781     0.289131   …  -0.0957613    0.0931928
 -0.502422  -0.234079    0.39433     -0.522318      -0.00397762  -0.101862 
 -0.462491  -0.176845    0.0754888   -0.429057      -0.313079    -0.148029 
 -0.742798   0.0321182  -0.326396    -0.0551339      0.0965478    0.10826  
 -0.666056  -0.0370985  -0.256263    -0.153015      -0.405469    -0.190896 

In [46]:
bests = show_bests(we, wes') 

20x20 Array{Any,2}:
 "Long"                 0.63  "``"          0.71  …  0.66  "."           0.84
 "."                    0.62  "always"      0.69     0.64  "yesterday."  0.69
 "Tiger"                0.59  "never"       0.68     0.62  "there."      0.67
 "Black"                0.58  "not"         0.66     0.62  "later."      0.66
 "Marvel"               0.57  "even"        0.66     0.57  "said."       0.64
 "Kobe"                 0.57  "probably"    0.66  …  0.57  "thinks"      0.64
 "Tory"                 0.56  "simply"      0.65     0.56  "..."         0.64
 "..."                  0.56  "hardly"      0.65     0.56  "put"         0.62
 "Jaffna"               0.55  "now"         0.64     0.53  "down."       0.62
 "packed"               0.55  "we"          0.63     0.53  "again."      0.62
 "genetically-altered"  0.55  "they"        0.63  …  0.53  "before."     0.61
 "Emergency"            0.54  "only"        0.62     0.53  "?"           0.6 
 ":"                    0.54  "Campbell"    

In [47]:
bests[1,1:2:end]

1x10 Array{Any,2}:
 "Long"  "``"  "."  "--"  "260-megawatt"  "."  "looked"  "."  ";"  "."

In [44]:
m1 = [ -2.95807685e-01,  -1.56413664e-04,  -1.35392015e-01,
         -2.70674817e-02,   1.50730582e-01,  -1.43258247e-01,
         -1.80694148e-02,  -4.22560263e-01,  -7.63571698e-02,
         -2.50450740e-01,  -4.22523800e-02,   2.03915750e-01,
         -1.46009625e-01,   1.30386263e-01,   5.58378064e-03,
          1.33240760e-01,  -1.55507962e-01,   6.55887035e-02,
          2.42313779e-01,   2.19345795e-01,   6.19273446e-02,
         -3.13966609e-02,   1.22540443e-02,   2.49910294e-01,
          4.37676729e-02,  -3.05363399e-01,   1.23334295e-01,
          1.31547411e-01,  -1.06140538e-01,   1.30608969e-01,
          8.56167043e-02,   3.84014344e-02,  -2.48863037e-01,
          2.43432432e-01,   1.66721571e-01,   3.06094270e-02,
          7.02452735e-02,  -3.21758337e-01,   1.91290475e-01,
          1.86490454e-01,  -1.77033321e-01,  -4.02242349e-02,
          3.32063074e-01,   1.43411687e-01,   1.94464260e-01,
          6.01609478e-02,   5.09167512e-02,  -2.62185423e-01,
         -5.81407855e-02,   4.69568881e-02]

show_best(we,m1)

20x2 Array{Any,2}:
 "--"           0.81
 "as"           0.76
 "away"         0.76
 "."            0.76
 "here"         0.76
 ";"            0.75
 "for"          0.73
 "out"          0.72
 "down"         0.72
 "just"         0.71
 "alone"        0.71
 "over"         0.71
 "because"      0.71
 "established"  0.71
 "but"          0.71
 "calls"        0.71
 "shared"       0.7 
 "delivered"    0.7 
 "within"       0.7 
 "today"        0.7 